# Koog A2A Server Notebook

This notebook demonstrates how to set up and run an Agent-to-Agent (A2A) server using the Koog library. The server exposes an AI agent that can generate jokes.

## 1. Dependencies

This cell declares the necessary dependencies for the A2A server. It includes libraries for the Koog agents, the A2A server features, and the HTTP JSON-RPC transport.

In [3]:
// show kernel version
"""
Kotlin Jupyter kernel version: ${notebook.kernelVersion} (Update in Settings > Tools > Kotlin Notebooks > Kernel version)
Java Runtime Environment version: ${notebook.jreInfo.javaVersion}
"""


Kotlin Jupyter kernel version: 0.15.2.706.dev1 (Update in Settings > Tools > Kotlin Notebooks > Kernel version)
Java Runtime Environment version: 21


In [2]:
USE {
    repositories {
        mavenCentral()
    }
    dependencies {
        implementation("ai.koog:koog-agents-jvm:0.5.2")
        implementation("ai.koog:agents-features-a2a-server-jvm:0.5.2")
        implementation("ai.koog:a2a-server-jvm:0.5.2")
        implementation("ai.koog:a2a-transport-server-jsonrpc-http-jvm:0.5.2")
    }
}
notebook.dependencyManager.currentBinaryClasspath.map { it.toPath().fileName }.joinToString("\n")

lib-0.15.2-706-1.jar
protocol-api-0.15.2-706-1.jar
kotlin-reflect-2.2.20.jar
api-0.15.2-706-1.jar
annotations-13.0.jar
kotlinx-serialization-json-jvm-1.9.0.jar
kotlin-stdlib-2.2.20.jar
kotlin-script-runtime-2.2.20.jar
slf4j-api-2.0.17.jar
kotlinx-serialization-core-jvm-1.9.0.jar

In [2]:
import ai.koog.a2a.exceptions.A2AUnsupportedOperationException
import ai.koog.a2a.model.APIKeySecurityScheme
import ai.koog.a2a.model.AgentCapabilities
import ai.koog.a2a.model.MessageSendParams
import ai.koog.a2a.model.Role
import ai.koog.a2a.model.TaskIdParams
import ai.koog.a2a.model.TaskState
import ai.koog.a2a.model.TaskStatus
import ai.koog.a2a.model.TaskStatusUpdateEvent
import ai.koog.a2a.model.TextPart
import ai.koog.a2a.server.A2AServer
import ai.koog.a2a.server.agent.AgentExecutor
import ai.koog.a2a.server.session.RequestContext
import ai.koog.a2a.server.session.SessionEventProcessor
import ai.koog.a2a.model.AgentCard
import ai.koog.a2a.model.AgentInterface
import ai.koog.a2a.model.AgentProvider
import ai.koog.a2a.model.AgentSkill
import ai.koog.a2a.model.HTTPAuthSecurityScheme
import ai.koog.a2a.model.In
import ai.koog.a2a.model.Task
import ai.koog.a2a.model.TransportProtocol
import ai.koog.a2a.transport.server.jsonrpc.http.HttpJSONRPCServerTransport
import ai.koog.agents.a2a.core.A2AMessage
import ai.koog.agents.a2a.core.MessageA2AMetadata
import ai.koog.agents.a2a.core.toA2AMessage
import ai.koog.agents.a2a.core.toKoogMessage
import ai.koog.agents.a2a.server.feature.A2AAgentServer
import ai.koog.agents.core.agent.AIAgent
import ai.koog.agents.core.agent.config.AIAgentConfig
import ai.koog.agents.core.dsl.builder.strategy
import ai.koog.agents.core.tools.ToolRegistry
import ai.koog.prompt.dsl.prompt
import ai.koog.prompt.executor.clients.openai.OpenAIClientSettings
import ai.koog.prompt.executor.clients.openai.OpenAILLMClient
import ai.koog.prompt.executor.clients.openai.OpenAIModels
import ai.koog.prompt.executor.llms.SingleLLMPromptExecutor
import ai.koog.prompt.llm.LLMCapability
import ai.koog.prompt.llm.LLMProvider
import ai.koog.prompt.llm.LLModel
import ai.koog.prompt.message.Message
import io.ktor.server.cio.CIO
import kotlinx.coroutines.Deferred
import kotlinx.coroutines.Dispatchers
import kotlinx.coroutines.runBlocking
import kotlinx.coroutines.delay
import kotlinx.datetime.Clock
import java.util.UUID
import kotlin.uuid.ExperimentalUuidApi
import kotlin.uuid.Uuid

## 2. Agent Creation

This cell defines the AI agent that will power our A2A server. The `createAgent` function configures and returns an `AIAgent` instance.

The agent uses a `SingleLLMPromptExecutor` with an `OpenAILLMClient`. This means it will use an OpenAI-compatible LLM to generate responses.

The `apiToken` is a placeholder and the `OpenAIClientSettings` point to a local LLM server. The follow example is using local LM studio to host `openai/gpt-oss-20b` model.

In [3]:
// Define the LLM model to be used by the agent
// You can install https://lmstudio.ai/ and run local model in your machine
val model = LLModel(
    provider = LLMProvider.OpenAI,
    id = "openai/gpt-oss-20b",
    capabilities = listOf(
        LLMCapability.Tools,
        LLMCapability.ToolChoice,
        LLMCapability.Speculation,
        LLMCapability.Schema.JSON.Basic,
        LLMCapability.Schema.JSON.Standard,
        LLMCapability.Document,
        LLMCapability.Completion,
        LLMCapability.MultipleChoices,
        LLMCapability.OpenAIEndpoint.Completions,
        LLMCapability.OpenAIEndpoint.Responses,
    ),
    contextLength = 128_000,
    maxOutputTokens = 8_000,
)

// Function to create a new AI agent
fun createAgent(): AIAgent<String, String> = AIAgent(
    promptExecutor = SingleLLMPromptExecutor(
        OpenAILLMClient(
            "apiToken", // Placeholder API token
            OpenAIClientSettings("http://localhost:1234/") // URL of the local LM Studio server
        )
    ),
    systemPrompt = "You are an assistant helping user to generate jokes", // System prompt for the agent
    llmModel = model,
)

## 3. Agent Executor

The `AgentExecutor` is the core component that handles incoming requests and interacts with the AI agent. The `SimpleJokeAgentExecutor` class implements the `AgentExecutor` interface.

In the `execute` method, it performs the following steps:
1. Retrieves the user's message from the request context.
2. Saves the incoming message to the message storage.
3. Loads the conversation history.
4. Creates a prompt with the conversation history.
5. Calls the AI agent to get a response.
6. Saves the agent's response to the message storage.
7. Sends the response back to the client.

In [4]:
class SimpleJokeAgentExecutor : AgentExecutor {
    @OptIn(ExperimentalUuidApi::class)
    override suspend fun execute(context: RequestContext<MessageSendParams>, eventProcessor: SessionEventProcessor) {
        val userMessage = context.params.message

        if (context.task != null || !userMessage.referenceTaskIds.isNullOrEmpty()) {
            throw A2AUnsupportedOperationException("This agent doesn't support tasks")
        }

        // Save incoming message to the current context
        context.messageStorage.save(userMessage)

        // Load all messages from the current context
        val contextMessages = context.messageStorage.getAll().map { it.toKoogMessage() }

        val prompt = prompt("") {
            // Append current message context
            messages(contextMessages)
        }

        // Get a response from the LLM
        val responseMessage = createAgent().run(prompt.toString())
            .let {
                ai.koog.agents.a2a.core.A2AMessage(
                    role = Role.Agent,
                    messageId = Uuid.random().toString(),
                    contextId = context.contextId,
                    parts = listOf(TextPart(it)),
                )
            }

        // Save the response to the current context
        context.messageStorage.save(responseMessage)

        // Reply with message
        eventProcessor.sendMessage(responseMessage)
    }
}

## 4. Agent Card

The `AgentCard` provides metadata about the agent to other agents and clients. It includes information like the agent's name, description, communication settings, capabilities, and skills.

This agent card defines the following:
- **Basic Identity**: Name, description, and version.
- **Communication Settings**: The server URL and preferred transport protocol.
- **Capabilities**: Streaming, push notifications, and state transition history.
- **Content Type Support**: Supported input and output MIME types.
- **Skills**: A list of skills the agent possesses, in this case, `joke-generation`.

In [5]:
val agentCard = AgentCard(
    // Basic Identity
    name = "Joke maker",
    description = "AI agent specialized in telling joke",
    version = "2.1.0",
    protocolVersion = "0.3.0",

    // Communication Settings
    url = "http://localhost:9000/a2a",
    preferredTransport = TransportProtocol.JSONRPC,

    // Capabilities Declaration
    capabilities = AgentCapabilities(
        streaming = true,              // Support real-time responses
        pushNotifications = true,      // Send async notifications
        stateTransitionHistory = true  // Maintain task history
    ),

    // Content Type Support
    defaultInputModes = listOf("text/plain", "text/markdown", "image/jpeg"),
    defaultOutputModes = listOf("text/plain", "text/markdown", "application/json"),

    // Define available security schemes
    securitySchemes = mapOf(),

    // Specify security requirements (logical OR of requirements)
    security = listOf(),

    // Enable extended card for authenticated users
    supportsAuthenticatedExtendedCard = true,

    // Skills/Capabilities
    skills = listOf(
        AgentSkill(
            id = "joke-generation",
            name = "Joke Generation",
            description = "Generate joke based on content",
            tags = listOf("joke"),
            examples = listOf(
                "Joke about apple",
            )
        ),
    ),
)

println(agentCard)

AgentCard(protocolVersion=0.3.0, name=Joke maker, description=AI agent specialized in telling joke, url=http://localhost:9000/a2a, preferredTransport=TransportProtocol(value=JSONRPC), additionalInterfaces=null, iconUrl=null, provider=null, version=2.1.0, documentationUrl=null, capabilities=AgentCapabilities(streaming=true, pushNotifications=true, stateTransitionHistory=true, extensions=null), securitySchemes={}, security=[], defaultInputModes=[text/plain, text/markdown, image/jpeg], defaultOutputModes=[text/plain, text/markdown, application/json], skills=[AgentSkill(id=joke-generation, name=Joke Generation, description=Generate joke based on content, tags=[joke], examples=[Joke about apple], inputModes=null, outputModes=null, security=null)], supportsAuthenticatedExtendedCard=true, signatures=null)


## 5. Server Initialization and Start

This cell initializes and starts the A2A server.

1. An `A2AServer` instance is created with the `SimpleJokeAgentExecutor` and the `agentCard`.
2. An `HttpJSONRPCServerTransport` is created to handle communication over HTTP.


In [6]:
// Create the A2A server instance
val a2aServer = A2AServer(
    agentExecutor = SimpleJokeAgentExecutor(),
    agentCard = agentCard
)

// Create the HTTP JSON-RPC transport
val transport = HttpJSONRPCServerTransport(a2aServer)
val port = 9000


3. The `transport.start()` method starts the server on port 9000, listening on the `/a2a` path.

Note: The Kotlin notebook interruption is not working when `wait = true`, thus using `while(true){..}` as a replacement

In [18]:
// Start the server
runBlocking {
    println("Running a2a server at http://localhost:$port")
    transport.start(
        agentCard = agentCard,
        engineFactory = CIO,           // Ktor engine (CIO, Netty, Jetty)
        port = port,                   // Server port
        path = "/a2a",                 // API endpoint path
        wait = false                   // Do not block until the server stops, setting true will not work in Kotlin Notebook interruption
    )
    println("Server started. Remeber to stop the server using the following cell or use restart the whole kernel")
}

Running a2a server at http://localhost:9000
Server started. Remeber to stop the server using the following cell or use restart the whole kernel


## 6. Stop the Server

This cell stops the A2A server. It's important to run this cell when you are finished to release the port.

In [19]:
// Stop the server
runBlocking {
    println("Stopping a2a server at http://localhost:$port")
    transport.stop()
    println("Stopped")
}

Stopping a2a server at http://localhost:9000
Stopped
